## Метрики моделей классификации 

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from IPython.display import display
from sklearn.datasets import load_iris

from sklearn.pipeline import Pipeline
from sklearn.preprocessing import MinMaxScaler, StandardScaler, RobustScaler, Normalizer
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV, KFold 
from sklearn.neighbors import KNeighborsClassifier 

##### Визуал контейнера ###
# from sklearn import set_config
# set_config(display="diagram")


import sklearn
#pip install --upgrade scikit-learn
print(sklearn.__version__)


1.1.2


# Зачем нам нужна метрика?

**Представим себе следующую задачу:** в больницу обращается 90% здоровых людей и 10% больных. Нам выделили квадрилион денег, что бы мы в 90% случаях определяли болен или здоров пациент.

Не долго думая мы создали модель самого глубокого обучения в которой, каждому обратившемуся мы говорили  - "вы здоровы"! 

**Профффит!!!** Мы вполнили ТЗ.

Я думаю, не нужно объяснять, что сделали мы всем очень плохо, особенно если мы сами прийдем в данное лечебное учреждение ...

Давайте разберемся какие метрики существую для решения задач классификации и как их расчитывать и самое главное как их применять в реальных задачах.

**Спойлер** Как не смешно, но выбор правильной метрики это задача бизнеса, любому датасатанисту, который хочет добиться успеха в своей области нужно достаточно глубоко вникать в реальную задачу (и часто не слушать окружающих его "эффективных" менеджеров), что бы понять какую метрику нужно использовать для решения этой задачи. Часто готовых метрик будет не хватать. В данном курсе, в основном, мы будем учиться искать из условия задачи, какая стандартная метрика лучше всего подходит для решения задачи. 


Для простоты и удобства сначала, будут построены метрики для бинарной классификации, а затем покажем отличия для многоклассовой классификации.

# Стандартные метрики бинарной классиикации

[`sklearn.metrics`](https://scikit-learn.org/stable/modules/model_evaluation.html#classification-metrics)

Для понимания метрик в терминах ошибок классификации запишим матрицу ошибок (*confusion matrix*). Обратите внимание на расположение истиных и ложных строк и столбцов - оно отлично от принятого в математической статистике.


|                     | y_pred = 0 | y_pred = 1          |
| ----------- |:------------:| ---------------:|
| **y_true = 0**       | True Negative (**TN**)     | False Positive (**FP**) |
| **y_true = 1**       | False Negative (**FN**)    | True Positive (**TP**)  |

Почему именно такая запись? Потому что, именно так выводится данная матрица в библиотеках python, не будем создавать путаницу в теории и практике.

## Правильность (Accuracy)

[`sklearn.metrics.accuracy_score`](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.accuracy_score.html?highlight=accuracy#sklearn.metrics.accuracy_score)

Интуитивно понятной, очевидной и редко используемой метрикой является правильность (accuracy) — доля правильных ответов нашего алгоритма:
$$ accuracy = \frac{TN+TP}{TN+FP +FN+TP}$$

Увы эта мерика бесполезна в задачах с неравными классами, например как в нашей с вами задаче о больнице (0- здоровые, 1 - больные, кстати почему так, а не наоборот?):

|                     | y_pred = 0 | y_pred = 1          |
| ----------- |:------------:| ---------------:|
| **y_true = 0**       | 90     | 10 |
| **y_true = 1**       | 0    |  0 |

$$ accuracy = \frac{90+0}{90+10 +0+0}=\frac{90}{100}=0.9$$

А теперь модифицируем задачу:

|                     | y_pred = 0 | y_pred = 1          |
| ----------- |:------------:| ---------------:|
| **y_true = 0**       | 81     | 9 |
| **y_true = 1**       | 1    |  9 |

$$ accuracy = \frac{81+9}{81+9 +9+1}=\frac{90}{100}=0.9$$

Результат одинаков, а для больных находим лучше :)

## Точность (Precision)

[`sklearn.metrics.precision_score`](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.precision_score.html#sklearn.metrics.precision_score)

Давайте посмотрим метрику точности:

$$ precision = \frac{TP}{TP+FP}$$

|                     | y_pred = 0 | y_pred = 1          |
| ----------- |:------------:| ---------------:|
| **y_true = 0**       | True Negative (**TN**)     | <span style="color:red">False Positive (**FP**)</span> |
| **y_true = 1**       | False Negative (**FN**)    | <span style="color:red">True Positive (**TP**)</span>  |


Данная метрика покажет насколько точно мы предсказываем класс 1, есть ли у нас мусор в предсказанном классе 1 из класса 0. Ошибки в самом классе 1 нас не интересуют.

|                     | y_pred = 0 | y_pred = 1          |
| ----------- |:------------:| ---------------:|
| **y_true = 0**       | 90     | 10 |
| **y_true = 1**       | 0    |  0 |

$$ precision = \frac{0}{0+10} = 0$$

|                     | y_pred = 0 | y_pred = 1          |
| ----------- |:------------:| ---------------:|
| **y_true = 0**       | 81     | 9 |
| **y_true = 1**       | 1    |  9 |

$$ precision = \frac{9}{9+9}=\frac{9}{18}=0.5$$


Точность можно интерпретировать как долю объектов, названных классификатором положительными и при этом **действительно** являющимися положительными.

## Полнота (recall)

[`sklearn.metrics.recall_score`](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.recall_score.html?highlight=sklearn+metrics+recall_score#sklearn.metrics.recall_score)

$$ recall = \frac{TP}{TP+TN}$$

|                     | y_pred = 0 | y_pred = 1          |
| ----------- |:------------:| ---------------:|
| **y_true = 0**       | True Negative (**TN**)     | False Positive (**FP**)</span> |
| **y_true = 1**       | <span style="color:red">False Negative (**FN**)</span>     | <span style="color:red">True Positive (**TP**)</span>  |


Данная метрика покажет насколько полно мы предсказываем класс 1 и какую часть в этом предсказании мы теряем. Важное замечание - мусор появившийся от ошибочных предсказаний 0 как 1 нас не волнует. 

|                     | y_pred = 0 | y_pred = 1          |
| ----------- |:------------:| ---------------:|
| **y_true = 0**       | 90     | 10 |
| **y_true = 1**       | 0    |  0 |

$$ precision = \frac{0}{0} = 0$$

|                     | y_pred = 0 | y_pred = 1          |
| ----------- |:------------:| ---------------:|
| **y_true = 0**       | 81     | 9 |
| **y_true = 1**       | 1    |  9 |

$$ precision = \frac{9}{1+9}=\frac{9}{10}=0.9$$

Данная метрика демонстрирует способность алгоритма обнаруживать позитивный класс.

### Промежуточный вывод:

Если мы решаем задачу об эффективной больнице, то тот класс, который мы ищем как целевой показатель должен иметь номер 1 (в бинарной классификации). Именно под его поиск "заточены" метрики. Очевидно, что в нашей задаче - это поиск больных пациентов.

- метрика правильности не подходит, так как не дает картину хорошего качества поиска больных в выборке.
- метрика точности не подходит так как мы не знаем сколько больных мы пропустили.
- метрика полноты позволит понять насколько хорошо мы находим больных и минимизация FN позволит улучшить качество обнаружения больных, но при этом (с точки зрания поиска больного), мы можем не беспокоится о неверных диагнозах (FP) - потом до обследуем :), поэтому это лучшая метрика, однако есть НО ...

Однако для страховой медецины, бюджета и т.п. это не оцень удачный подход, мы слишком наразумно тратим финансы. Поэтому нам нужна метрика, которая сбалансирует FN и  FP при поиске TP

## Метрика $F_1$

[`sklearn.metrics.f1_score`](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.f1_score.html?highlight=sklearn+metrics+f1_score#sklearn.metrics.f1_score)

Метрика $F_1$ представляет собой балас между точностью и полнотой в поиске положительного класса. Расчитывается следующим образом:

$$F_1 = 2 \frac{precission\cdot recall}{precission+recall}=\frac{2}{\frac{1}{precission}+\frac{1}{recall}}=
\frac{TP}{TP+\frac{1}{2}(FP+FN)}
$$

|                     | y_pred = 0 | y_pred = 1          |
| ----------- |:------------:| ---------------:|
| **y_true = 0**       | True Negative (**TN**)     | <span style="color:red">False Positive (**FP**)</span> |
| **y_true = 1**       | <span style="color:red">False Negative (**FN**)</span>    | <span style="color:red">True Positive (**TP**)</span>  |


|                     | y_pred = 0 | y_pred = 1          |
| ----------- |:------------:| ---------------:|
| **y_true = 0**       | 90     | 10 |
| **y_true = 1**       | 0    |  0 |

$$ F_1 = \frac{0}{0+0.5(0+10)} = 0$$

|                     | y_pred = 0 | y_pred = 1          |
| ----------- |:------------:| ---------------:|
| **y_true = 0**       | 81     | 9 |
| **y_true = 1**       | 1    |  9 |

$$ precision = \frac{9}{9+0.5(1+9)}=\frac{9}{14} = 0.64$$

С точки зрания страховой результат явно не очень ... слишком много расходов не нужных расходов ...

## Средне гармоническая метрика $F_\beta$

[`sklearn.metrics.fbeta_score`](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.fbeta_score.html?highlight=sklearn+metrics+fbeta_score#sklearn.metrics.fbeta_score)

$\beta$ в данном случае определяет вес точности в метрике, и при $\beta = 1$ это среднее гармоническое (с множителем 2, чтобы в случае precision = 1 и recall = 1 иметь $F_1 = 1$)
F-мера достигает максимума при полноте и точности, равными единице, и близка к нулю, если один из аргументов близок к нулю.

$$F_\beta = (1+\beta^2) \frac{precission\cdot recall}{\beta^2 precission+recall}\\
F_\beta  = \frac{(1+\beta^2)TP}{(1+\beta^2)TP+\beta^2FN+FP)}
$$

Параметр бета определяет вес полноты в ценке. $\beta < 1$ придает больший вес точности, если же $\beta > 1$ по вес распределяетя в сторону полноты:
- $\beta \rightarrow 0$ учитывается только точность; 
- $\beta \rightarrow +\infty$ только полнота.

## Логистическая функция потерь

[`sklearn.metrics.log_loss`](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.log_loss.html?highlight=sklearn+metrics+log_loss#sklearn.metrics.log_loss)

Интуитивно можно представить минимизацию `log_loss` как задачу максимизации `accuracy` путем штрафа за неверные предсказания. Однако необходимо отметить, что `log_loss` крайне сильно штрафует за уверенность классификатора в неверном ответе.

Данная метрика нечасто выступает в бизнес-требованиях, но часто — в задачах на kaggle.

$$ log\_loss = \frac{1}{n}\sum_{i=1}^n\left(y_i \ln(p_i)+(1-y_i)\ln(1-p_i)\right)$$

где $y_i \in {0,1}$ - значение класса из `y_true`, $p_i = Pr(y_i=1)$ вероятность классификации $y_i$ как 1. 

Важный коментарий! Возможно применять если алгоритм реализует `predict_proba` - вероятность классификации, как например [`KNeighborsClassifier`](https://scikit-learn.org/stable/modules/generated/sklearn.neighbors.KNeighborsClassifier.html#sklearn.neighbors.KNeighborsClassifier.predict_proba)

## ROC AUC, PR AUC

**Важный термин!** AUC это сокращение от Area Under Curve - площадь под кривой. Употреблять отдельно без указания конкретной кривой - странно, но увы вчтречается постоянно, жаргон ...

Мы только, что говорили, что алгоритмы (большинство) предсказывают `predict_proba` - вероятность классификации заданного класса. Вопрос а какой порог можно поставить чтобы отдель эти классы и что от этого меняется?

Естественным и близким кажется порог, равный 0.5, но он не всегла является оптимальным, например при дисбалансе классов.

Одним из способов оценить модель в целом, не привязываясь к конкретному порогу, является ROC AUC — площадь (Area Under Curve) под кривой ошибок (Receiver Operating Characteristic curve ). Данная кривая представляет из себя линию от (0,0) до (1,1) в координатах True Positive Rate (TPR) и False Positive Rate (FPR):

$$ TPR = \frac{TP}{TP+FN}$$
$$ FPR = \frac{FP}{TP+TN}$$

TPR нам уже известна, это полнота, а FPR показывает, какую долю из объектов negative класса алгоритм предсказал неверно.

В идеальном случае, когда классификатор не делает ошибок (FPR = 0, TPR = 1) мы получим площадь под кривой, равную единице. 

В противном случае, когда классификатор случайно выдает вероятности классов, AUC-ROC будет стремиться к 0.5, так как классификатор будет выдавать одинаковое количество TP и FP.

![Пример из интернетов](https://hsto.org/r/w1560/web/299/157/fad/299157fad56a4ecca8f6b96b425bd38c.png)

Каждая точка на графике соответствует выбору некоторого порога. Площадь под кривой в данном случае показывает качество алгоритма (больше — лучше), кроме этого, важной является крутизна самой кривой — мы хотим максимизировать TPR, минимизируя FPR, а значит, наша кривая в идеале должна стремиться к точке (0,1).

ROC AUC оценивает устойчивочть алгоритма, так как позволяет оценить качество всех порогов сразу.  

Критерий AUC-ROC устойчив к несбалансированным классам (но: увы, не всё так однозначно) и может быть интерпретирован как вероятность того, что случайно выбранный positive объект будет проранжирован классификатором выше (будет иметь более высокую вероятность быть positive), чем случайно выбранный negative объект.

Рассмотрим следующую задачу: нам необходимо выбрать 100 релевантных документов из 1 миллиона документов. Мы намашинлернили два алгоритма:

**Алгоритм 1** возвращает 100 документов, 90 из которых релевантны. Таким образом,

$$ TPR = \frac{TP}{TP + FN} = \frac{90}{90 + 10} = 0.9$$

$$ FPR = \frac{FP}{FP + TN} = \frac{10}{10 + 999890} = 0.00001$$


**Алгоритм 2** возвращает 2000 документов, 90 из которых релевантны. Таким образом,

$$ TPR = \frac{TP}{TP + FN} = \frac{90}{90 + 10} = 0.9$$


$$ FPR = \frac{FP}{FP + TN} = \frac{1910}{1910 + 997990} = 0.00191$$


Скорее всего, мы бы выбрали первый алгоритм, который выдает очень мало FP на фоне своего конкурента. Но разница в FRP между этими двумя алгоритмами крайне мала — всего 0.0019. Это является следствием того, что ROC AUC измеряет долю FP относительно TN и в задачах, где нам не так важен второй (больший) класс, может давать не совсем адекватную картину при сравнении алгоритмов.

Для того чтобы поправить положение, вернемся к полноте и точности :

**Алгоритм 1**

$$ precision = \frac{TP}{TP + FP} = 90/(90 + 10) = 0.9 $$


$$ recall = \frac{TP}{TP + FN} = 90/(90 + 10) = 0.9 $$


**Алгоритм 2**

$$ precision = \frac{TP}{TP + FP} = \frac{90}{90 + 1910} = 0.045 $$


$$ recall = \frac{TP}{TP + FN} = \frac{90}{90 + 10} = 0.9 $$


Здесь уже заметна существенная разница между двумя алгоритмами — 0.855 в точности.

Precision и recall также используют для построения кривой PR AUC находят площадь под ней. При этом наилучшая точка находится в в точке с координатами (1, 1)

![Пример из интернетов](https://hsto.org/r/w1560/web/887/356/914/8873569147c24b49a2b7fa9ae883f20f.png)

**Важная ссылка!** За подробностями о взаимоотношениях ROC AUC и PR AUC можно обратиться [сюда](https://pages.cs.wisc.edu/~jdavis/davisgoadrichcamera2.pdf).

## Метрики имбалансных классов

**Стоит только начать и сразу можно потеряться**

### Специализированная библиотека

Специализированная [библиотека](https://imbalanced-learn.org/stable/index.html) для понимания степени, меры и глубины проблем ...

Увы на это у нас нехватит времени ...

### Стандартный способ

[`sklearn.metrics.balanced_accuracy_score`](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.balanced_accuracy_score.html#sklearn.metrics.balanced_accuracy_score)

Можно использовать для многоклассовой классификации. Для бинарной нужно внимательно смотреть на условия конкретно задачи.

$$ balanced\_accuracy = \frac{1}{2}\left(\frac{TP}{TP+FN}+\frac{TN}{TN+FP}\right) $$

Важная ссылка! [Метрики и примеры](https://scikit-learn.org/stable/modules/model_evaluation.html)




## Мультиклассовая классификация и метрики

Пройдемся только по самым основным понятиям, более глубоко будет время вернемся в конце курса.

исходник [тут](https://towardsdatascience.com/micro-macro-weighted-averages-of-f1-score-clearly-explained-b603420b292f)

![Картинка1](https://miro.medium.com/max/1400/1*FbxvE-0qJb6L-GEQ2RCyNg.png)

Посчитали

![Картинка2](https://miro.medium.com/max/1400/1*er8MJ85T4lY0zZoRGqMwxQ.png)

Вспомним как считается остальное:

![Картинка3](https://miro.medium.com/max/1400/1*1TGTF8chlODIBMIvU6xUUg.png)

`Macro averaging` единое среднее для классов f1
![Картинка4](https://miro.medium.com/max/1400/1*vLFtzVb2MaFcs2cECuTO5Q.png)

`Micro averaging` сумма TP, нормированная на TP+1/2(FP+FN) для каждого класса. Важно!  'micro avg' == 'accuracy' если учитываются все классы, если же указать только подмножество классов, то в выводе появится строчка с 'micro avg' вместо 'accuracy'.
![Картинка5](https://miro.medium.com/max/1400/1*ordMpMsZ5WR6q1Wsd8B24Q.png)

`Weighted Average` средневзвешенная оценка с учетом долей классов
![Картинка6](https://miro.medium.com/max/1400/1*0SN1NHpAz154YOjgBLCCYw.png)


Как считается можно посмотреть [тут](https://www.educative.io/answers/what-is-the-difference-between-micro-and-macro-averaging)

В чем же отличие macro, micro и weighted:

- "macro" просто вычисляет среднее значение показателей, придавая каждому классу одинаковый вес. В задачах, где редкие классы важны, макро-усреднение может быть средством выделения их значимости. С другой стороны, предположение, что все классы одинаково важны, часто неверно, так что макро-усреднение будет чрезмерно подчеркивать обычно низкую значимость для редкого класса.

- "weighted" учитывает дисбаланс классов, вычисляя среднее значение показателей, в которых оценка каждого класса взвешивается по его присутствию в истинной выборке данных.

- "micro" дает каждой паре строка данных соотнесенной с классом равный вклад в общую метрику (за исключением результата взвешивания выборки). Вместо того, чтобы суммировать метрику для каждого класса, мы суммируем составные части метрик (TP, FN, FP) для каждого класса, для расчета общего частного. Микро-усреднение может быть предпочтительным в мультиклассовой классификации, когда классы с наибольшим количеством строк (данных) следует игнорировать.

## Вишенка на торте

Еще про метрики классификации - [крайне полезная таблица](https://en.wikipedia.org/wiki/Template:Diagnostic_testing_diagram)